In [1]:
import importlib
import layers.tied
importlib.reload(layers)
import tensorflow as tf
#tf.enable_eager_execution()
print(tf.__version__)

1.12.0


In [18]:
def eval_tensor(t):
    return tf.Session().run(t)

def tensor2const(t):
    return tf.constant(eval_tensor(t))


Rx = tensor2const(tf.random.uniform([1,100]))
Ry = tensor2const(tf.random.uniform([1,50]))
TDxy = layers.tied.TiedDenseLayer(units=50) 
TDyx = layers.tied.TiedDenseLayer(units=100)



In [31]:
#with tf.name_scope("tied") as scope:
#X = tf.placeholder(tf.float32, shape=(None, 100))
#Y = tf.placeholder(tf.float32, shape=(None, 50))

assert(not tf.executing_eagerly())
inputs = tf.keras.Input(shape=(100,))  ##tensor=X)
outputs = TDxy(inputs)
Xreconstruct = TDyx(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
#model = tf.keras.models.Sequential([TDxy])
def loss_func(y0, y1):
    return (tf.losses.mean_squared_error(y0, y1) *0.5+ 
            0.5 * tf.losses.mean_squared_error(inputs, Xreconstruct))
    
optimizer = tf.train.MomentumOptimizer(use_nesterov=True, learning_rate=0.001, momentum=0.8)
##train_step = optimizer.minimize(loss)
model.compile(optimizer, loss=loss_func) ##'mean_squared_error')

model.fit(Rx, Ry, epochs=50,steps_per_epoch=1)

Epoch 1/50
1/1 [==============================] - 1s 555ms/step - loss: 0.4154
Epoch 2/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4147
Epoch 3/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4137
Epoch 4/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4126
Epoch 5/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4113
Epoch 6/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4099
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 0.4084
Epoch 8/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4068
Epoch 9/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4052
Epoch 10/50
1/1 [==============================] - 0s 3ms/step - loss: 0.4035
Epoch 11/50
1/1 [==============================] - 0s 4ms/step - loss: 0.4018
Epoch 12/50
1/1 [==============================] - 0s 4ms/step - loss: 0.4001
Epoch 13/50
1/1 [==============================] - 0s 7ms/step - loss: 

In [42]:
W = TDxy.get_weights()[0]
#print(W.shape)
print(Rx[0].shape)
print (tf.matmul(Rx , W ).shape)
eval_tensor(tf.nn.l2_loss(tf.matmul(Rx , W ) - Ry))
#( W * tf.transpose(Rx)).shape
#tf.nn.l2_loss(tf.matmul(Rx,W) - Ry).numpy() / 50
#print(eval_tensor(Rx))


(100,)
(1, 50)


9.00128

<tf.Tensor: id=568, shape=(1, 50), dtype=float32, numpy=
array([[-9.7477329e-01, -8.2208133e-01, -9.6124542e-01,  3.7618971e-04,
        -1.0394732e+00,  1.7865970e+00,  4.8999241e-01,  2.4646701e-01,
         1.0472475e+00, -2.9509002e-01, -2.6252449e-01, -7.6602143e-01,
        -7.8683645e-01, -7.7154410e-01,  3.2458159e-01, -4.6965221e-01,
        -1.6701154e+00, -6.6593868e-01, -4.5266682e-01,  1.9592406e-01,
        -2.9994237e-01,  1.4538711e-01,  4.0084203e-03, -5.3980872e-03,
        -2.4856642e-01, -2.5537807e-02, -1.6502229e+00,  4.1029409e-01,
         4.1868094e-01, -5.6240731e-01, -3.7862131e-01,  8.3970487e-02,
         4.8118734e-01, -5.6614824e-02, -2.4418516e-02,  2.7894941e-01,
        -3.7845142e-02,  3.0466071e-01,  4.8948762e-01, -8.0523741e-01,
        -4.0582216e-01,  6.4324540e-01,  1.5459983e-01,  6.6791065e-02,
        -1.2349693e+00,  9.8919946e-01,  2.9284969e-01,  7.3838133e-01,
         6.8548733e-01,  1.6602484e+00]], dtype=float32)>

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

onehot = lambda y: tf.one_hot(y, depth=10)
flat = lambda x : tf.reshape(x, (-1, x.shape[1] * x.shape[2]))
(x_train, y_train_labels),(x_test, y_test_labels) = mnist.load_data()
y_train = onehot(y_train_labels)
y_test = onehot(y_test_labels)
# flat may not be needed if input is a tf.dense layer
x_train = flat(x_train)
x_test = flat(x_test)
print(x_train.shape, x_test.shape)